In [1]:
%load_ext autoreload
%autoreload 1
%aimport ecg_get_data
%aimport Models
%aimport train_test_validat
%aimport self_attention
%aimport ECGplot
import Models 
from train_test_validat import *
from self_attention import *
from  ecg_get_data import *
import matplotlib.pyplot as plt

import torch
import torch.utils.data as Data
import random

import time
import os
import gc

random_seed = 2
torch.manual_seed(random_seed)    # reproducible
torch.cuda.manual_seed_all(random_seed)
random.seed(random_seed)
np.random.seed(random_seed)

time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime()) 

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

data_path =  './npy_ECG/' #路径
lable_path = './label.npy'
model_path = './model/'+time_str
log_path = './log/'+  time_str


EcgChannles_num = 12
EcgLength_num = 5000
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda:1


In [3]:
x = load_data(data_path,EcgChannles_num=EcgChannles_num,EcgLength_num=EcgLength_num)
y = load_label(lable_path)

100%|██████████| 1698/1698 [00:08<00:00, 189.67it/s]


In [4]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y = train_test_split(x,y,train_size=0.8,random_state = random_seed,shuffle = True,stratify=y)
del x
del y
gc.collect()
print("         HTN     NHTN ")
print("train: %5d   %5d" % (train_y.sum(),len(train_y)-train_y.sum()))
print("test : %5d   %5d" % (test_y.sum(),len(test_y)-test_y.sum()))

         HTN     NHTN 
train:   558     800
test :   140     200


In [6]:
train_x ,train_y = sliding_window(train_x ,train_y,2000,20)
test_x ,test_y = sliding_window(test_x ,test_y,2000,20)
print("         HTN     NHTN ")
print("train: %5d   %5d" % (train_y.sum(),len(train_y)-train_y.sum()))
print("test : %5d   %5d" % (test_y.sum(),len(test_y)-test_y.sum()))

100


100%|██████████| 1358/1358 [00:01<00:00, 1052.23it/s]


42098 42098
100


100%|██████████| 340/340 [00:00<00:00, 1232.73it/s]

10540 10540
         HTN     NHTN 
train: 17298   24800
test :  4340    6200


In [7]:
BATCH_SIZE = 128
FOLDS = 1
EPOCHS = 5000  
PATIENCE = 100
LR = 0.0001

In [8]:
from torch.utils.tensorboard import SummaryWriter   
os.makedirs(model_path, exist_ok=True)
writer = SummaryWriter(log_path)
from torchsummary import summary

In [15]:
#NET = [Models.resnet18(input_channels=12, inplanes=32, num_classes=2) for i in range(FOLDS)]
NET = [Models.CNN_ATT_Mark() for i in range(FOLDS)]
print(summary(NET[0], (EcgChannles_num,2000), device="cpu"))
#writer.add_graph(NET[0],torch.rand([1,EcgChannles_num,EcgLength_num]))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1             [-1, 12, 2000]              24
            Conv1d-2             [-1, 32, 2000]           4,256
              ReLU-3             [-1, 32, 2000]               0
       BatchNorm1d-4             [-1, 32, 2000]              64
            Conv1d-5             [-1, 32, 2000]           5,152
              ReLU-6             [-1, 32, 2000]               0
       BatchNorm1d-7             [-1, 32, 2000]              64
         MaxPool1d-8              [-1, 32, 500]               0
            Conv1d-9              [-1, 32, 500]           1,056
             ReLU-10              [-1, 32, 500]               0
      BatchNorm1d-11              [-1, 32, 500]              64
          Dropout-12              [-1, 32, 500]               0
           Conv1d-13              [-1, 32, 500]           1,056
             ReLU-14              [-1, 

In [16]:
from torch.optim.lr_scheduler import CosineAnnealingLR

for fold in range(FOLDS):
    
    early_stopping = EarlyStopping(PATIENCE, verbose=True, model_path=model_path, delta=0)
    #train_dataset,valid_dataset = get_k_fold_dataset(fold=int(fold+1),x = train_x,y=train_y,k=FOLDS,random_seed = random_seed)
    valid_dataset  = load_numpy_dataset_to_tensor_dataset(test_x,test_y)
    train_dataset = load_numpy_dataset_to_tensor_dataset(train_x,train_y)
    train_dataloader = Data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    valid_dataloader = Data.DataLoader(dataset=valid_dataset, batch_size=BATCH_SIZE, shuffle=True)

    NET[fold].to(DEVICE)
    optimizer  = torch.optim.Adam(NET[fold].parameters(), lr=LR,weight_decay=1e-2)  
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10, gamma=0.1)
    criterion = torch.nn.CrossEntropyLoss()   
    #等间隔调整学习率
    #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max = 32)
    best_loss = 3
    for epoch in range(1,EPOCHS):
        time_all=0
        start_time = time.time()
        train_loss,train_acc = train_model(train_dataloader, NET[fold], criterion, optimizer,DEVICE) # 训练模型
        #scheduler.step() # 学习率迭代

        time_all = time.time()-start_time
        validate_loss,validate_acc = test_model(valid_dataloader,criterion,NET[fold],DEVICE) # 测试模型
        writer.add_scalars(main_tag=str(fold)+'_Loss',tag_scalar_dict={'train': train_loss,'validate': validate_loss},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_Accuracy',tag_scalar_dict={'train': train_acc,'validate': validate_acc},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_LearningRate',tag_scalar_dict={'LR': optimizer.state_dict()['param_groups'][0]['lr']},global_step=epoch)

        print('- Epoch: %d - Train_loss: %.5f - Train_acc: %.5f - Val_loss: %.5f - Val_acc: %5f - T_Time: %.3f' %(epoch,train_loss,train_acc,validate_loss,validate_acc,time_all))
        print('当前学习率：%f' %optimizer.state_dict()['param_groups'][0]['lr'])

        if validate_loss < best_loss:
            best_loss = validate_loss
            print('Find better model in Epoch {0}, saving model.'.format(epoch))
            #torch.save(NET[fold],  model_path+'/best_model_' + str(fold) + '.pt')  # 保存最优模型
        else:
            scheduler.step() # 学习率迭代
        #是否满足早停法条件
        if(early_stopping(validate_loss,NET[fold])):
            print("Early stopping")
            break

    print('Fold %d Training Finished' %(fold+1))
    torch.cuda.empty_cache()# 清空显卡cuda
print('Training Finished')

- Epoch: 1 - Train_loss: 0.64167 - Train_acc: 0.62880 - Val_loss: 0.65791 - Val_acc: 0.603402 - T_Time: 71.758
当前学习率：0.000100
Find better model in Epoch 1, saving model.
Validation loss decreased (inf --> 0.657906).  Saving model ...
                    --------------------------------------------------

- Epoch: 2 - Train_loss: 0.61490 - Train_acc: 0.66645 - Val_loss: 0.65783 - Val_acc: 0.589078 - T_Time: 70.867
当前学习率：0.000100
Find better model in Epoch 2, saving model.
Validation loss decreased (0.657906 --> 0.657827).  Saving model ...
                    --------------------------------------------------

- Epoch: 3 - Train_loss: 0.59630 - Train_acc: 0.69149 - Val_loss: 0.65325 - Val_acc: 0.605456 - T_Time: 70.166
当前学习率：0.000100
Find better model in Epoch 3, saving model.
Validation loss decreased (0.657827 --> 0.653251).  Saving model ...
                    --------------------------------------------------

- Epoch: 4 - Train_loss: 0.58525 - Train_acc: 0.70671 - Val_loss: 0.6811

In [ ]:
from torch.optim.lr_scheduler import CosineAnnealingLR

for fold in range(FOLDS):
    
    early_stopping = EarlyStopping(PATIENCE, verbose=True, model_path=model_path, delta=0)
    #train_dataset,valid_dataset = get_k_fold_dataset(fold=int(fold+1),x = train_x,y=train_y,k=FOLDS,random_seed = random_seed)
    valid_dataset  = load_numpy_dataset_to_tensor_dataset(test_x,test_y)
    train_dataset = load_numpy_dataset_to_tensor_dataset(train_x,train_y)
    train_dataloader = Data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    valid_dataloader = Data.DataLoader(dataset=valid_dataset, batch_size=BATCH_SIZE, shuffle=True)

    NET[fold].to(DEVICE)
    optimizer  = torch.optim.Adam(NET[fold].parameters(), lr=LR,weight_decay=1e-2)  
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10, gamma=0.1)
    criterion = torch.nn.CrossEntropyLoss()   
    #等间隔调整学习率
    #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max = 32)
    best_loss = 3
    for epoch in range(1,EPOCHS):
        time_all=0
        start_time = time.time()
        train_loss,train_acc = train_model(train_dataloader, NET[fold], criterion, optimizer,DEVICE) # 训练模型
        #scheduler.step() # 学习率迭代

        time_all = time.time()-start_time
        validate_loss,validate_acc = test_model(valid_dataloader,criterion,NET[fold],DEVICE) # 测试模型
        writer.add_scalars(main_tag=str(fold)+'_Loss',tag_scalar_dict={'train': train_loss,'validate': validate_loss},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_Accuracy',tag_scalar_dict={'train': train_acc,'validate': validate_acc},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_LearningRate',tag_scalar_dict={'LR': optimizer.state_dict()['param_groups'][0]['lr']},global_step=epoch)

        print('- Epoch: %d - Train_loss: %.5f - Train_acc: %.5f - Val_loss: %.5f - Val_acc: %5f - T_Time: %.3f' %(epoch,train_loss,train_acc,validate_loss,validate_acc,time_all))
        print('当前学习率：%f' %optimizer.state_dict()['param_groups'][0]['lr'])

        if validate_loss < best_loss:
            best_loss = validate_loss
            print('Find better model in Epoch {0}, saving model.'.format(epoch))
            #torch.save(NET[fold],  model_path+'/best_model_' + str(fold) + '.pt')  # 保存最优模型
        else:
            scheduler.step() # 学习率迭代
        #是否满足早停法条件
        if(early_stopping(validate_loss,NET[fold])):
            print("Early stopping")
            break

    print('Fold %d Training Finished' %(fold+1))
    torch.cuda.empty_cache()# 清空显卡cuda
print('Training Finished')

- Epoch: 1 - Train_loss: 0.64167 - Train_acc: 0.62880 - Val_loss: 0.65791 - Val_acc: 0.603402 - T_Time: 71.758
当前学习率：0.000100
Find better model in Epoch 1, saving model.
Validation loss decreased (inf --> 0.657906).  Saving model ...
                    --------------------------------------------------

- Epoch: 2 - Train_loss: 0.61490 - Train_acc: 0.66645 - Val_loss: 0.65783 - Val_acc: 0.589078 - T_Time: 70.867
当前学习率：0.000100
Find better model in Epoch 2, saving model.
Validation loss decreased (0.657906 --> 0.657827).  Saving model ...
                    --------------------------------------------------

- Epoch: 3 - Train_loss: 0.59630 - Train_acc: 0.69149 - Val_loss: 0.65325 - Val_acc: 0.605456 - T_Time: 70.166
当前学习率：0.000100
Find better model in Epoch 3, saving model.
Validation loss decreased (0.657827 --> 0.653251).  Saving model ...
                    --------------------------------------------------

- Epoch: 4 - Train_loss: 0.58525 - Train_acc: 0.70671 - Val_loss: 0.6811

# ------Display the attention value-----

# -----For Leads -----

In [ ]:
test_model_path = "./model/20220907_091538/best_model_0.pt"
test_model = torch.load(test_model_path).to(DEVICE)
test_x = x[0:1698]
test_y = x[0:1698]
test_x = MAX_MIN_normalization_by_feactures(test_x)
test_x = torch.FloatTensor(test_x)  #turn numpy to tensor
test_y = torch.LongTensor(test_y)
test_dataset = Data.TensorDataset(test_x, test_y)
test_dataloader = Data.DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)
test_model.eval()

test_model.eval()
attention_value_timestep = np.zeros(EcgChannles_num,)
for i,data in enumerate(test_dataloader,0):
    inputs,labels = data[0].to(DEVICE),data[1].to(DEVICE)
    outputs = test_model(inputs)
    #print(outputs)
    _,pred = outputs.max(1) # 求概率最大值对应的标签
    #print("the label :{labels},pred is {pred}".format(labels=labels[0],pred=pred[0]))
    attention_value = test_model.attention_value
    attention_value_timestep += (((attention_value.to('cpu'))[0]).detach().numpy()).sum(axis=0)
plt.figure(figsize=(5,5))
plt.stem(np.arange(0,len(attention_value_timestep)), attention_value_timestep)
plt.xticks(np.arange(0,len(attention_value_timestep)),['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'])
plt.ylabel("Sum of attention value")
plt.show()

In [ ]:
print(summary(Models.CNN_ATT7(), (EcgChannles_num,EcgLength_num), device="cpu"))

In [ ]:
x = np.random.randint(10, high=309, size=[12], dtype='l')